# hw3 - ensembles

## 1 Подготовка данных

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D

import category_encoders as ec
from sklearn.preprocessing import OrdinalEncoder
import numpy as np
from tqdm.notebook import tqdm

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold, KFold

from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.svm import SVC, SVR
from sklearn.tree import DecisionTreeClassifier

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, accuracy_score

Загрузите и предобработайте данные (по своему усмотрению) из hw1

In [2]:
data = pd.read_csv('./data.csv')

In [3]:
X = data.drop(columns=["G3"])
y = data["G3"]

## 2 Обоснуйте выбор слабых (базовых) алгоритмов

Выбрал слабые алгоритме основываясь на алгоритмах из первой домашке, которые хорошо себя в целом показали + решил добавить DecisionTreeClassifier, так как тоже себя хорошо показывает на этих данных

## 3 Постройте решение на основе подхода Blending

Правила:
- Нужно использовать вероятности
- Предложите что-то лучше, чем брать среднее от предсказаний моделей (оценивать уверенность алгоритмов, точности и т.д.)
- Заставьте базовые алгоритмы быть некорелированными
- Добавьте рандома (например, стройте ваши алгоритмы на разных выборках, по разному предобрабатывайте данные или применяйте для разных признаков соответствующие алгоритмы ... )
- Проявите смекалку
- Цель: метрика MSE на тесте меньше 10

In [4]:
def classifier_models():
    models = {
        "DecisionTreeClassifier": DecisionTreeClassifier(),
        "knn": KNeighborsClassifier(),
        "svm": SVC(probability=True),
        "lr" : LogisticRegression()
    }
    meta_model = SVC()
    return models, meta_model

In [5]:
def classifier_grids():
    grid = {
        "DecisionTreeClassifier": {
            'max_depth': np.arange(1, 2), 
            "splitter": ["best", "random"],
            "criterion": ["gini", "entropy", "log_loss"]
        },
        "knn": {
            "n_neighbors": np.arange(10, 30),
            'metric': ['euclidean', 'manhattan', 'minkowski']
        },
        "svm": {
            "class_weight": ["balanced"],
            "kernel": ["linear", "poly", "rbf"],
            "C": np.linspace(0.001, 10, 20),
            "max_iter": [50, 500, 1000, 10000]
        },
        "lr": {
            "solver": ['saga', 'lgbfs'],
            "C": [1, 2, 3],
            "penalty": ["l1","l2"],
            "max_iter": [100, 200]
        }
    }

    meta_grid = {
        "class_weight": ["balanced", "uniform"],
        "kernel": ["linear", "poly", "rbf"],
        "C": np.linspace(0.001, 10, 20)
    }
    return grid, meta_grid

In [6]:
def regressor_models():
    models = {
        "knn": KNeighborsRegressor(),
        "svr": SVR(),
        "lr" : ElasticNet()
    }

    meta_model = SVR()
    return models, meta_model

def regressor_grids():
    grid = {
        "knn": {
            "n_neighbors": np.arange(1, 60)
        },
        "svr": {
            "kernel": ["linear", "poly", "rbf"],
            "C": np.linspace(0.001, 10, 20)
        },
        "lr": {
            "fit_intercept": [True]
        }
    }

    meta_grid = {
        "kernel": ["linear", "poly", "rbf"],
        "C": np.linspace(0.001, 10, 20)
    }
    return grid, meta_grid

In [7]:
def train_base_models(data, models, grid, verbose=False):
    for model_name in tqdm(models.keys()):
        if verbose:
            print(f"Model: {model_name}")
        models[model_name] = fit_predict_CV(data["X"], data["y"], models[model_name], grid[model_name], verbose=verbose)

    return models

In [8]:
def fit_predict_CV(X, y, model_type, params_grid, verbose=False):
    model_cv = GridSearchCV(model_type, params_grid, cv=KFold(), refit=True)
    model_cv.fit(X, y)

    if verbose:
        print("Best hyperparameters: ", model_cv.best_params_)
        print("Best score: ", model_cv.best_score_)

    return model_cv.best_estimator_

def train_base_models(data, models, grid, verbose=False):
    for model_name in tqdm(models.keys()):
        if verbose:
            print("Model:",{model_name})
        models[model_name] = fit_predict_CV(data["X"], data["y"], models[model_name], grid[model_name], verbose=verbose)

    return models

In [9]:
def predict_base_models(X, models, models_type=None):
    if models_type is None:
        raise ValueError("models_type must be non None")
    predictions = None
    if models_type == "cls":
        for model in models.values():
            pred = model.predict_proba(X)
            if predictions is None:
                predictions = pred
            else:
                predictions = np.hstack([predictions, pred])
    if models_type == "reg":
         for model in models.values():
            pred = model.predict(X)
            if predictions is None:
                predictions = pred[:, None]
            else:
                predictions = np.hstack([predictions, pred[:, None]])

    return predictions

def train_meta_model(data, models, meta_model, grid, models_type=None, verbose=False):
    X_train_meta_model = predict_base_models(data["X"], models, models_type)
    return fit_predict_CV(X_train_meta_model, data["y"], meta_model, grid, verbose=verbose)

In [10]:
def predict_blending(data, models, meta_model, models_type=None):
    models_preds = predict_base_models(data["X"], models, models_type)
    predictions = meta_model.predict(models_preds)
    return predictions

In [11]:
x_train, x_val, y_train, y_val = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=42)
x_train, x_test, y_train, y_test = train_test_split(x_train, y_train, test_size=0.2, shuffle=True, random_state=42)

scaler = StandardScaler()
scaler.fit(x_train)

data_train = {
    "X": scaler.transform(x_train),
    "y": y_train
}
data_val = {
    "X": scaler.transform(x_val),
    "y": y_val
}
data_test = {
    "X": scaler.transform(x_test),
    "y": y_test
}

In [12]:
models, meta_model = classifier_models()
grid, meta_model_grid = classifier_grids()

In [13]:
def warn(*args, **kwargs):
    pass

import warnings
warnings.warn = warn

In [14]:
best_models = train_base_models(data_train, models, grid, verbose=True)
trained_meta_model = train_meta_model(
    data_val,
    best_models,
    meta_model,
    meta_model_grid,
    models_type="cls"
)

  0%|          | 0/4 [00:00<?, ?it/s]

Model: {'DecisionTreeClassifier'}
Best hyperparameters:  {'criterion': 'entropy', 'max_depth': 1, 'splitter': 'random'}
Best score:  0.1896551724137931
Model: {'knn'}
Best hyperparameters:  {'metric': 'euclidean', 'n_neighbors': 22}
Best score:  0.1724137931034483
Model: {'svm'}
Best hyperparameters:  {'C': 4.737368421052632, 'class_weight': 'balanced', 'kernel': 'rbf', 'max_iter': 50}
Best score:  0.16551724137931037
Model: {'lr'}
Best hyperparameters:  {'C': 1, 'max_iter': 100, 'penalty': 'l2', 'solver': 'saga'}
Best score:  0.13793103448275862


In [15]:
predictions = np.round(predict_blending(data_test, best_models, trained_meta_model, models_type="cls"))
accuracy = accuracy_score(data_test["y"], np.round(predictions))
mse = mean_squared_error(data_test["y"], np.round(predictions))

print(f"Accuracy: {accuracy:.3}")
print(f"MSE:      {mse:.5}")

Accuracy: 0.106
MSE:      8.937


## 4 Постройте решение на основе подхода Stacking

In [16]:
from sklearn.ensemble import StackingClassifier
from sklearn.pipeline import make_pipeline

Правила:
- Реализуйте пайплайн обучения и предсказания (например, sklearn.pipeline или класс)
- Проведите оптимизацию пайплайна
- Оцените вклад каждого базового алгоритма в итоговое предсказание
- Цель: метрика MSE на тесте меньше 10

In [17]:
models = classifier_models()[0]
estimators = []
for model_name in models.keys():
    model_pipeline = make_pipeline(
        StandardScaler(),
        models[model_name]
    )
    estimators += [(model_name, model_pipeline)]

folds = StratifiedKFold(n_splits=3, shuffle=True)

clf = StackingClassifier(
    estimators=estimators,
    final_estimator=LogisticRegression(),
    cv=folds
)

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=1)

clf = clf.fit(x_train, y_train)
predictions = clf.predict(x_test)

accuracy = accuracy_score(y_test, np.round(predictions))
mse = mean_squared_error(y_test, np.round(predictions))

print(f"Accuracy: {accuracy:.3}")
print(f"MSE:      {mse:.5}")

Accuracy: 0.198
MSE:      8.6044


## * Доп задание (не обязательно, но решение будет поощряться)

Правила:
- Постройте несколько сильных алгоритмов разного класса (это может быть бустинг, нейросеть, ансамбль слабых алгоритмов, алгоритм на статистике, что придумаете)
- Реализуйте "управляющий" алгоритм, который на основе входных данных будет выбирать, какой из  сильных алгоритмов запустить (не на основе их работы, а именно на основе данных)